In [1]:
import polars as pl

In [2]:
train_target = pl.read_parquet('../data/train_target.parquet', use_pyarrow=True)
test_target = pl.read_parquet('../data/test_target_b.parquet', use_pyarrow=True)

In [ ]:
# транзакции
train_emb_df = pl.to_csv('../data/train_trans_emb_v8.csv', index=False)
val_emb_df = pl.to_csv('../data/val_trans_emb_v8.csv', index=False)
test_emb_df = pl.to_csv('../data/test_trans_emb_v8.csv', index=False)

In [4]:
# гео
train_geo_emb_df1 = pl.read_csv('../data/train1_geo_emb_v2.csv')
train_geo_emb_df2 = pl.read_csv('../data/train2_geo_emb_v2.csv')
train_geo_emb_df3 = pl.read_csv('../data/train3_geo_emb_v2.csv')
train_geo_emb_df4 = pl.read_csv('../data/train4_geo_emb_v2.csv')
val_geo_emb_df = pl.read_csv('../data/val_geo_emb_v8.csv')
test_geo_emb_df = pl.read_csv('../data/test_geo_emb_v8.csv')

In [5]:
# диалоги
train_dial_emb_df = pl.read_csv('../data/train_dial_emb_v8.csv')
val_dial_emb_df = pl.read_csv('../data/val_dial_emb_v8.csv')
test_dial_emb_df = pl.read_csv('../data/test_dial_emb_v8.csv')

In [6]:
train_geo_emb_df = pl.concat([train_geo_emb_df1, train_geo_emb_df2, train_geo_emb_df3, train_geo_emb_df4])

In [13]:
train_geo_emb_df = train_geo_emb_df.filter(train_geo_emb_df['month'] == '2023-01-31')
train_dial_emb_df = train_dial_emb_df.filter(train_dial_emb_df['month'] == '2023-01-31')
train_target = train_target.filter(train_target['mon'] == '2023-01-31')

In [33]:
len(test_target['client_id'].unique())

140488

In [16]:
emb_cols = ['emb_0000', 'emb_0001', 'emb_0002', 'emb_0003', 'emb_0004', 'emb_0005', 'emb_0006', 'emb_0007', 'emb_0008', 'emb_0009', 'emb_0010', 'emb_0011', 'emb_0012', 'emb_0013', 'emb_0014', 'emb_0015', 'emb_0016', 'emb_0017', 'emb_0018', 'emb_0019', 'emb_0020', 'emb_0021', 'emb_0022', 'emb_0023', 'emb_0024', 'emb_0025', 'emb_0026', 'emb_0027', 'emb_0028', 'emb_0029', 'emb_0030', 'emb_0031', 'emb_0032', 'emb_0033', 'emb_0034', 'emb_0035', 'emb_0036', 'emb_0037', 'emb_0038', 'emb_0039', 'emb_0040', 'emb_0041', 'emb_0042', 'emb_0043', 'emb_0044', 'emb_0045', 'emb_0046', 'emb_0047', 'emb_0048', 'emb_0049', 'emb_0050', 'emb_0051', 'emb_0052', 'emb_0053', 'emb_0054', 'emb_0055', 'emb_0056', 'emb_0057', 'emb_0058', 'emb_0059', 'emb_0060', 'emb_0061', 'emb_0062', 'emb_0063', 'emb_0000_geo', 'emb_0001_geo', 'emb_0002_geo', 'emb_0003_geo', 'emb_0004_geo', 'emb_0005_geo', 'emb_0006_geo', 'emb_0007_geo', 'emb_0008_geo', 'emb_0009_geo', 'emb_0010_geo', 'emb_0011_geo', 'emb_0012_geo', 'emb_0013_geo', 'emb_0014_geo', 'emb_0015_geo', 'emb_0016_geo', 'emb_0017_geo', 'emb_0018_geo', 'emb_0019_geo', 'emb_0020_geo', 'emb_0021_geo', 'emb_0022_geo', 'emb_0023_geo', 'emb_0024_geo', 'emb_0025_geo', 'emb_0026_geo', 'emb_0027_geo', 'emb_0028_geo', 'emb_0029_geo', 'emb_0030_geo', 'emb_0031_geo', 'emb_0032_geo', 'emb_0033_geo', 'emb_0034_geo', 'emb_0035_geo', 'emb_0036_geo', 'emb_0037_geo', 'emb_0038_geo', 'emb_0039_geo', 'emb_0040_geo', 'emb_0041_geo', 'emb_0042_geo', 'emb_0043_geo', 'emb_0044_geo', 'emb_0045_geo', 'emb_0046_geo', 'emb_0047_geo', 'emb_0048_geo', 'emb_0049_geo', 'emb_0050_geo', 'emb_0051_geo', 'emb_0052_geo', 'emb_0053_geo', 'emb_0054_geo', 'emb_0055_geo', 'emb_0056_geo', 'emb_0057_geo', 'emb_0058_geo', 'emb_0059_geo', 'emb_0060_geo', 'emb_0061_geo', 'emb_0062_geo', 'emb_0063_geo']

In [15]:
train = train_target\
    .join(train_dial_emb_df.drop('', 'target_1', 'target_2', 'target_3', 'target_4', 'month'), on='client_id', how='left', coalesce=True, suffix='_dial')\
        .join(train_geo_emb_df.drop('month'), on='client_id', how='left', coalesce=True, suffix='_geo')
train = train.to_pandas().dropna(subset=emb_cols, how='all').fillna(0)

In [28]:
test = test_dial_emb_df.drop('', 'target_1', 'target_2', 'target_3', 'target_4')\
        .join(test_geo_emb_df, on='client_id', how='left', coalesce=True, suffix='_geo')

test = test.to_pandas().dropna(subset=emb_cols, how='all').fillna(0)

In [34]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


c:\Users\Aleksandr\Documents\python_apps\lct-2024\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


c:\Users\Aleksandr\Documents\python_apps\lct-2024\.venv\lib\site-packages\lightautoml\text\tokenizer.py:21: UserWarning: 'nltk' - package isn't installed
  warnings.warn("'nltk' - package isn't installed")
c:\Users\Aleksandr\Documents\python_apps\lct-2024\.venv\lib\site-packages\lightautoml\transformers\text.py:22: UserWarning: 'gensim' - package isn't installed
  warnings.warn("'gensim' - package isn't installed")
c:\Users\Aleksandr\Documents\python_apps\lct-2024\.venv\lib\site-packages\lightautoml\ml_algo\dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")


In [38]:
oof_pred = model.fit_predict(train[emb_cols + ['target_1', ]], roles={'target': 'target_1'})

In [39]:
model1 = model
oof_pred1 = oof_pred

In [40]:
params = {'timeout': 1_000,}
params['task_type'] = 'binary'

In [ ]:
model1 = TabularAutoML(
    task = Task(params['task_type']),
    gpu_ids=None,
    timeout = params['timeout'],
    cpu_limit = 8,
    reader_params = {
        'n_jobs': 8, 
        'cv': 3, 
        'random_state': 0,
        })
oof_pred1 = model1.fit_predict(train[emb_cols + ['target_1', ]], roles={'target': 'target_1'})

In [41]:
model2 = TabularAutoML(
    task = Task(params['task_type']),
    gpu_ids=None,
    timeout = params['timeout'],
    cpu_limit = 8,
    reader_params = {
        'n_jobs': 8, 
        'cv': 3, 
        'random_state': 0,
        })
oof_pred2 = model2.fit_predict(train[emb_cols + ['target_2', ]], roles={'target': 'target_2'})

In [42]:
model3 = TabularAutoML(
    task = Task(params['task_type']),
    gpu_ids=None,
    timeout = params['timeout'],
    cpu_limit = 8,
    reader_params = {
        'n_jobs': 8, 
        'cv': 3, 
        'random_state': 0,
        })
oof_pred3= model3.fit_predict(train[emb_cols + ['target_3', ]], roles={'target': 'target_3'})

In [43]:
model4 = TabularAutoML(
    task = Task(params['task_type']),
    gpu_ids=None,
    timeout = params['timeout'],
    cpu_limit = 8,
    reader_params = {
        'n_jobs': 8, 
        'cv': 3, 
        'random_state': 0,
        })
oof_pred4 = model4.fit_predict(train[emb_cols + ['target_4', ]], roles={'target': 'target_4'})

In [46]:
target_1 = model1.predict(test)
target_2 = model2.predict(test)
target_3 = model3.predict(test)
target_4 = model4.predict(test)

In [48]:
test['target_1'] = target_1.data[:, 0]
test['target_2'] = target_2.data[:, 0]
test['target_3'] = target_3.data[:, 0]
test['target_4'] = target_4.data[:, 0]

In [49]:
test[['client_id', 'target_1', 'target_2', 'target_3', 'target_4']].to_csv('submission.csv')